## Clustering

**Disclaimer: If you do "Run All", let everything load first, and dont interact with the maps in the clustering part without everything being loaded.**

In [ ]:
# Import all necessary libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import geoplot as gplt
import contextily as cx

# also import these "new" libraries 
# Note: you may have to download and add them to your environment (using e.g. 'conda install -c conda-forge folium')
# !important! Install this version of plotly=5.10.0 or else some maps and animations may not render correctly
import plotly.express as px
from haversine import haversine, Unit

# import the necessary libraries for the machine learning models
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA

from shapely.geometry import Polygon, MultiPolygon
from h3 import h3

In [ ]:
# This is set, to display all the columns
pd.set_option('display.max_columns', None)

# Load data
taxi_df_clustering = pd.read_parquet('data/01_prep/prepared/taxi_data_prepared.gzip')
df_h3 = pd.read_csv('data/03_clustering/hexagons.csv').drop(columns="Unnamed: 0")

We will sample our data, since it will either take way too long to compute or crash the entire kernel.

In [ ]:
taxi_df_clustering = taxi_df_clustering.sample(200000, random_state=0)
taxi_df_clustering.reset_index(drop=True, inplace=True)

### General Data Prep
Before we can start with the clustering, we add addtional features to our clustering dataframe.

#### Additional information extracted from the timestamps

In [ ]:
#Additional Data of weekday, hour and month
taxi_df_clustering.loc[:,"weekday"] = taxi_df_clustering["trip_start_timestamp"].dt.weekday
taxi_df_clustering.loc[:,"start_hour"] = taxi_df_clustering["trip_start_timestamp"].dt.hour
taxi_df_clustering.loc[:,"end_hour"] = taxi_df_clustering["trip_end_timestamp"].dt.hour
taxi_df_clustering.loc[:,"month"] = taxi_df_clustering["trip_start_timestamp"].dt.month

In [ ]:
taxi_df_clustering

#### POI data

In [ ]:
taxi_df_clustering = taxi_df_clustering.merge(df_h3[['pickup_centroid_location','h3_6']], how="left", on="pickup_centroid_location")

In [ ]:
def get_n_poi_for_resolution(resolution="h3_6"):
    # get polygons from hexagon id
    df_hex = gpd.read_file('data/03_clustering/hexagons.csv', index_col=[0])
    df_hex = df_hex[resolution].to_frame()
    df_hex['geometry'] = df_hex.apply(lambda x: Polygon(h3.h3_to_geo_boundary(x[resolution], geo_json=True)), axis=1)
    df_hex = df_hex.set_geometry("geometry")
    df_hex.crs = "EPSG:4326"
    df_hex = df_hex.to_crs("EPSG:4326")
    
    # load point of interest
    poi = gpd.read_file('data/03_clustering/POI.geojson')
    
    # join
    poi_by_hex = gpd.sjoin(poi, df_hex, how='inner', predicate='within')
    
    return poi_by_hex.groupby(resolution).size().to_frame().reset_index().rename(columns={0:"n_poi"})

In [ ]:
def get_n_poi_for_resolution_and_amenity(resolution="h3_6"):
    # get polygons from hexagon id
    df_hex = gpd.read_file('data/03_clustering/hexagons.csv', index_col=[0])
    df_hex = df_hex[resolution].to_frame()
    df_hex['geometry'] = df_hex.apply(lambda x: Polygon(h3.h3_to_geo_boundary(x[resolution], geo_json=True)), axis=1)
    df_hex = df_hex.set_geometry("geometry")
    df_hex.crs = "EPSG:4326"
    df_hex = df_hex.to_crs("EPSG:4326")
    
    # load point of interest
    poi = gpd.read_file('data/03_clustering/POI.geojson')
    
    # join
    poi_by_hex = gpd.sjoin(poi, df_hex, how='inner', predicate='within').groupby([resolution, "amenity"]).size().to_frame().reset_index()
    
    # generate according dataframe with features
    poi_features = []
    for hexagon in poi_by_hex[resolution].unique():
        di = {resolution: hexagon}
        for amenity in poi_by_hex.amenity.unique():
            n = poi_by_hex[(poi_by_hex[resolution] == hexagon) & (poi_by_hex["amenity"] == amenity)][0].values
            di[amenity] = n[0] if n.size > 0 else 0
        poi_features.append(di)
    
    return pd.DataFrame(poi_features)

In [ ]:
poi_by_hex = get_n_poi_for_resolution()
poi_amenity_by_hex = get_n_poi_for_resolution_and_amenity()

In [ ]:
# Adding POI data based on the hexagons 
taxi_df_clustering = taxi_df_clustering.merge(poi_by_hex[['h3_6','n_poi']], how="left", on="h3_6")
taxi_df_clustering = taxi_df_clustering.merge(poi_amenity_by_hex[['h3_6','bar','pub','car_rental','clinic', 'ferry_terminal']], how="left", on="h3_6")
taxi_df_clustering['bar/pub'] = taxi_df_clustering['bar']+taxi_df_clustering['pub']

taxi_df_clustering

#### Distance to City center

As City Center, we will consider the central business district of the city, which is widely known as the Loop, the main section of Downtown Chicago (Wikipedia). This corresponds to the location of 41.881111 (Latitude), -87.629722 (Longtitude). We will investigate clustering trip types based on their pickup and dropoff distance to the city center.

In [ ]:
# Converting both pickup and dropoff centroid location to a geoseries
taxi_df_clustering['pickup_centroid_location'] = gpd.GeoSeries.from_wkt(taxi_df_clustering['pickup_centroid_location'])
taxi_df_clustering['dropoff_centroid_location'] = gpd.GeoSeries.from_wkt(taxi_df_clustering['dropoff_centroid_location'])

# Creating a GeoDataFrame 
taxi_geo_df_clustering = gpd.GeoDataFrame(taxi_df_clustering, geometry='pickup_centroid_location', crs=4326)

In [ ]:
# Extracting latitude and longitude of the pickup location
taxi_df_clustering['pickup_centroid_location_lat'] = taxi_geo_df_clustering['pickup_centroid_location'].y
taxi_df_clustering['pickup_centroid_location_lon'] = taxi_geo_df_clustering['pickup_centroid_location'].x

# Extracting latitude and longitude of the dropoff location
taxi_df_clustering['dropoff_centroid_location_lat'] = taxi_geo_df_clustering['dropoff_centroid_location'].y
taxi_df_clustering['dropoff_centroid_location_lon'] = taxi_geo_df_clustering['dropoff_centroid_location'].x

In [ ]:
# Calculating the distance between start coordinates and the city center
for i in range(0, len(taxi_df_clustering)):
    city_center = (41.881111, -87.629722)
    start_coordinates = (taxi_df_clustering.at[i, 'pickup_centroid_location_lat'], taxi_df_clustering.at[i, 'pickup_centroid_location_lon'])
    end_coordinates = (taxi_df_clustering.at[i, 'dropoff_centroid_location_lat'], taxi_df_clustering.at[i, 'dropoff_centroid_location_lon'])

    # Units need to be set to "Unit.MILES"
    taxi_df_clustering.at[i, 'distance_to_city_center_pickup'] = haversine(start_coordinates, city_center, unit=Unit.MILES)
    taxi_df_clustering.at[i, 'distance_to_city_center_dropoff'] = haversine(end_coordinates, city_center, unit=Unit.MILES)

In [ ]:
taxi_df_clustering

### Functions to use for Clustering
Next we create a number of functions to help with frequent Clustering Steps:

In [ ]:
# This function is used to standardize features
def scalingData(dataframe):
    newDataframe = dataframe.copy()
    scaler = StandardScaler()
    newDataframe[newDataframe.columns] = pd.DataFrame(scaler.fit_transform(newDataframe[newDataframe.columns]))
    return newDataframe

In [ ]:
# This function calculates the Loss per cluster amount and plots the result of it in the range of 0-10 on the x-axis
def calcAndPlotLossKM(clusterAmount, dataframe):
    k_max = clusterAmount

    clusters = []
    losses = []

    for k in range(k_max):
        model = KMeans(n_clusters=k+1, n_init=10)
        model.fit(dataframe)
        clusters.append(k+1)
        losses.append(model.inertia_)

    fig = plt.subplots(figsize=(12,7))
    plt.plot(clusters, losses)
    plt.ylabel("Loss")
    plt.xlabel("Number of clusters")
    plt.xlim([0,10])
    plt.grid(True)
    plt.show()

In [ ]:
# This function calculates for a defined amount of clusters KMeans on the given dataframe
def calcKMeans(numClusters, dataframe):
    result = KMeans(n_clusters=numClusters, n_init=10)
    result.fit(dataframe)

    dataframe['ClusterKM'] = result.predict(dataframe)

In [ ]:
# This function calculates for a defined amount of clusters GMM on the given dataframe
def calcGMM(numClusters, dataframe):
    gmm = GaussianMixture(n_components=numClusters).fit(dataframe)
    dataframe['ClusterGMM'] = gmm.predict(dataframe)

In [ ]:
# This function describes every KMeans or GMM cluster with the describe() function for the original dataframe
def describeData(originalDataframe, scaledDataframe, method):   
    if(method == 'KMeans'):
        for i in range(0, scaledDataframe['ClusterKM'].max()+1):
            display(originalDataframe[scaledDataframe['ClusterKM'] == i].describe())
    elif(method == 'GMM'):
        for i in range(0, scaledDataframe['ClusterGMM'].max()+1):    
            display(originalDataframe[scaledDataframe['ClusterGMM'] == i].describe())
    else:
        print('Error: The wrong method has been chosen. Either use "KMeans" or "GMM"!')

In [ ]:
# This function removes outliers for a set of columns, where values is n times larger than std
def removeOutliers(df, columns, n_std):
    for col in columns:
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean + (n_std * sd))]
        df = df.reset_index(drop=True)
        
    return df

## Clustering Trip/Customer Types

To cluster Trip/Customer Types we will use the following features:
 - Miles of every trip
 - Distance to city center for both the pickup and dropoff locations
 - The coordinates of both the pickup and dropoff locations
 - The weekday of the trips
 - The start hour of the trips

In [ ]:
# Feature selection by dropping unnecessary columns
taxi_df_clustering.drop(columns=['trip_seconds',
                                'trip_start_timestamp',
                                'trip_end_timestamp',
                                'pickup_census_tract',
                                'dropoff_census_tract',
                                'trip_total',
                                'pickup_centroid_location',
                                'dropoff_centroid_location',
                                'idle_seconds',
                                'end_hour',
                                'month',
                                'h3_6',
                                'n_poi',
                                'bar',
                                'pub',
                                'car_rental',
                                'clinic',
                                'ferry_terminal',
                                'bar/pub'], inplace=True)

After selecting our features, we will have a look at our final dataset.

In [ ]:
taxi_df_clustering.describe()

We can see a very high standard deviation for our "trip_miles", hence we will remove outliers 3 times bigger than standard deviation.

In [ ]:
# Remove outliers in dataset as impact on clustering is significant
taxi_df_clustering = removeOutliers(taxi_df_clustering,['trip_miles'],3)

taxi_df_clustering.describe()

Taking care of outliers for "trip_miles" is necessary, since they can have a huge impact on the clustering performance. Now, we can scale our data.

In [ ]:
# Scaling the data
taxi_df_clustering_scaled = scalingData(taxi_df_clustering)
taxi_df_clustering_scaled

### PCA
Since we have 9 features in our final dataframe, we use PCA to reduce the dimensionality of the dataset.

In [ ]:
evaluate_pca = PCA().fit(taxi_df_clustering_scaled)

In [ ]:
plt.figure(figsize = (10,8))
plt.plot(range(1,10), evaluate_pca.explained_variance_ratio_.cumsum(), marker ='o', linestyle='--')
plt.title('Explained Variance by Components')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')

As we want to keep the majority of variance explained by the components, we settle for 4 out of the 10 components which roughly preserve 80% of the variance.

In [ ]:
pca = PCA(n_components = 4).fit(taxi_df_clustering_scaled)

In [ ]:
scores_pca = pca.transform(taxi_df_clustering_scaled)

scores_pca

In [ ]:
taxi_df_clustering_scaled_pca = pd.DataFrame(scores_pca, columns=['Component 1','Component 2','Component 3','Component 4'])

taxi_df_clustering_scaled_pca

### Soft Clustering with Gaussian Mixture Model

To start with our soft clustering approach, we first calculate the loss with a hard clustering method (KMeans) to find out a possible number of clusters to pick.

In [ ]:
#Calculating the Loss with increasing number of Clusters
calcAndPlotLossKM(10, taxi_df_clustering_scaled_pca)

Based on this we can try to pick cluster between 3 and 6 clusters.

#### Soft clustering calculation

In [ ]:
# Copy for KMeans
taxi_df_clustering_gmm = taxi_df_clustering_scaled_pca.copy()

# Calculating GMM
calcGMM(3, taxi_df_clustering_gmm)

# Adding the clusters from GMM into the original dataframe for visualization purposes
taxi_df_clustering.loc[:, 'ClusterGMM'] = taxi_df_clustering_gmm['ClusterGMM']

taxi_df_clustering_gmm

In [ ]:
describeData(taxi_df_clustering.loc[:, taxi_df_clustering.columns != 'ClusterGMM'], taxi_df_clustering_gmm, 'GMM')

In [ ]:
#Plotting the Clustering in 3D
fig = plt.figure(figsize=(10, 10))

# Colors to map to clusters
colors = {0:'#377eb8', 1:'#ff7f00', 2:'#4daf4a'}
ax = fig.add_subplot(projection='3d')

ax.scatter(xs=taxi_df_clustering_gmm['Component 1'], ys=taxi_df_clustering_gmm['Component 2'], zs=taxi_df_clustering_gmm['Component 3'], c=taxi_df_clustering_gmm['ClusterGMM'].map(colors))

handles = [Line2D([0], [0], marker='o', color='w', markerfacecolor=v, label=k, markersize=8) for k, v in colors.items()]
ax.legend(title='Cluster', handles=handles, bbox_to_anchor=(1.05, 1), loc='upper left')

plt.title('Clustering Results GMM')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
ax.set_zlabel('Component 3')
ax.zaxis.labelpad = 0
plt.show()

In [ ]:
# Scatter Mapbox to display the clusters for the pickups in a map
fig_pickup_gmm = px.scatter_mapbox(taxi_df_clustering, 
                        lat="pickup_centroid_location_lat", lon="pickup_centroid_location_lon", 
                        animation_frame='ClusterGMM', opacity=1, 
                        #animation_group='start_station_name', 
                        zoom=9.7, height=1000, width=900, 
                        title='Locational Clusters based on the pickup locations (GMM)')
fig_pickup_gmm.update_geos(center=dict(lon=-87.629722, lat=41.881111))
fig_pickup_gmm.update_layout(mapbox_style="carto-positron")
fig_pickup_gmm.update_layout(
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
   font=dict(
            size=18
        ))
display(fig_pickup_gmm)

In [ ]:
# Scatter Mapbox to display the clusters for the dropoffs in a map
fig_dropoff_gmm = px.scatter_mapbox(taxi_df_clustering, 
                        lat="dropoff_centroid_location_lat", lon="dropoff_centroid_location_lon", 
                        animation_frame='ClusterGMM', opacity=1, 
                        #animation_group='start_station_name', 
                        zoom=9.7, height=1000, width=900, 
                        title='Locational Clusters based on the dropoff locations (GMM)')
fig_dropoff_gmm.update_geos(center=dict(lon=-87.629722, lat=41.881111))
fig_dropoff_gmm.update_layout(mapbox_style="carto-positron")
fig_dropoff_gmm.update_layout(
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
   font=dict(
            size=18
        ))
display(fig_dropoff_gmm)

### Hard Clustering with KMeans

We already calculated the loss for the initialization of our GMM clusters, so we do not need to do it here again. Same thing applies here as well, being 3 to 6 clusters as a choice.

In [ ]:
# Copy for KMeans
taxi_df_clustering_kmeans = taxi_df_clustering_scaled_pca.copy()

# Calculating KMeans
calcKMeans(3, taxi_df_clustering_kmeans)

# Adding the clusters from KMeans into the original dataframe for visualization purposes
taxi_df_clustering.loc[:, 'ClusterKM'] = taxi_df_clustering_kmeans['ClusterKM']

taxi_df_clustering_kmeans

In [ ]:
#Plotting the Clustering in 3D
fig = plt.figure(figsize=(10, 10))

# Colors to map to clusters
colors = {0:'#377eb8', 1:'#ff7f00', 2:'#4daf4a'}
ax = fig.add_subplot(projection='3d')

ax.scatter(xs=taxi_df_clustering_kmeans['Component 1'], ys=taxi_df_clustering_kmeans['Component 2'], zs=taxi_df_clustering_kmeans['Component 3'], c=taxi_df_clustering_kmeans['ClusterKM'].map(colors))

handles = [Line2D([0], [0], marker='o', color='w', markerfacecolor=v, label=k, markersize=8) for k, v in colors.items()]
ax.legend(title='Cluster', handles=handles, bbox_to_anchor=(1.05, 1), loc='upper left')

plt.title('Clustering Results KMeans')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
ax.set_zlabel('Component 3')
ax.zaxis.labelpad = 0
plt.show()

In [ ]:
describeData(taxi_df_clustering.loc[:, ~taxi_df_clustering.columns.isin(['ClusterGMM', 'ClusterKM'])], taxi_df_clustering_kmeans, 'KMeans')

In [ ]:
# Scatter Mapbox to display the clusters for the pickups (KMeans) in a map
fig_pickup_kmeans = px.scatter_mapbox(taxi_df_clustering, 
                        lat="pickup_centroid_location_lat", lon="pickup_centroid_location_lon", 
                        animation_frame='ClusterKM', opacity=1, 
                        #animation_group='start_station_name', 
                        zoom=9.7, height=1000, width=900, 
                        title='Locational Clusters based on the pickup locations (KMeans)')
fig_pickup_kmeans.update_geos(center=dict(lon=-87.629722, lat=41.881111))
fig_pickup_kmeans.update_layout(mapbox_style="carto-positron")
fig_pickup_kmeans.update_layout(
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
   font=dict(
            size=18
        ))
display(fig_pickup_kmeans)

In [ ]:
# Scatter Mapbox to display the clusters for the dropoffs (KMeans) in a map
fig_pickup_kmeans = px.scatter_mapbox(taxi_df_clustering, 
                        lat="dropoff_centroid_location_lat", lon="dropoff_centroid_location_lon", 
                        animation_frame='ClusterKM', opacity=1, 
                        #animation_group='start_station_name', 
                        zoom=9.7, height=1000, width=900, 
                        title='Locational Clusters based on the dropoff locations (KMeans)')
fig_pickup_kmeans.update_geos(center=dict(lon=-87.629722, lat=41.881111))
fig_pickup_kmeans.update_layout(mapbox_style="carto-positron")
fig_pickup_kmeans.update_layout(
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
   font=dict(
            size=18
        ))
display(fig_pickup_kmeans)

## Kernel density estimation

To find spatial hot spots for trip demand we use Kernel density estimation with a gaussian kernel. The bandwidth selection is done by a "rule of thumb", called the Scott´s Rule.

In [ ]:
# Load data
taxi_df_locations = pd.read_parquet('data/01_prep/prepared/taxi_data_prepared.gzip')
boroughs = gpd.read_file('data/03_clustering/Boundaries - City.geojson')

# Sampling the data, because otherwise it takes way too long to compute or the kernel crashes
taxi_df_locations = taxi_df_locations.sample(100000, random_state=0)
taxi_df_locations.reset_index(drop=True, inplace=True)

In [ ]:
# Convert both centroid locations to a geoseries
taxi_df_locations['pickup_centroid_location'] = gpd.GeoSeries.from_wkt(taxi_df_locations['pickup_centroid_location'])
taxi_df_locations['dropoff_centroid_location'] = gpd.GeoSeries.from_wkt(taxi_df_locations['dropoff_centroid_location'])

# Creating GeoDataFrame
taxi_geo_df_locations = gpd.GeoDataFrame(taxi_df_locations, geometry='pickup_centroid_location', crs=4326)

In [ ]:
# Levels correspond to iso-proportions of the density: e.g., 20% of the probability mass will lie below the contour drawn for 0.2.

# KDE plot without clipping the data points
levels = [0.2,0.4,0.6,0.8,1]

kde = gplt.kdeplot(
    taxi_geo_df_locations['pickup_centroid_location'], 
    figsize=(10,10),
    levels = levels,
    fill = True,
    cmap = 'viridis',
    alpha = 0.9,
    legend = True,
    projection=gplt.crs.WebMercator()
)

cx.add_basemap(kde)

In [ ]:
# KDE plot with clipping the data points
levels = [0.2,0.4,0.6,0.8,1]

kde = gplt.kdeplot(
    taxi_geo_df_locations['pickup_centroid_location'], 
    figsize=(10,10),
    levels = levels,
    fill = True,
    cmap = 'viridis',
    alpha = 0.9,
    projection=gplt.crs.WebMercator(),
    clip=boroughs.geometry
)

cx.add_basemap(kde)

In [ ]:
# Source: https://towardsdatascience.com/from-kernel-density-estimation-to-spatial-analysis-in-python-64ddcdb6bc9b

# Extracting the calculated polygons for better visualization purposes
level_polygons = []
i = 0
for col in kde.collections:
    paths = []
    # Loop through all polygons that have the same intensity level
    for contour in col.get_paths(): 
        # Create a polygon for the countour
        # First polygon is the main countour, the rest are holes
        for ncp,cp in enumerate(contour.to_polygons()):
            x = cp[:,0]
            y = cp[:,1]
            new_shape = Polygon([(i[0], i[1]) for i in zip(x,y)])
            if ncp == 0:
                poly = new_shape
            else:
                # Remove holes, if any
                poly = poly.difference(new_shape)

        # Append polygon to list
        paths.append(poly)
    # Create a MultiPolygon for the contour
    multi = MultiPolygon(paths)
    # Append MultiPolygon and level as tuple to list
    level_polygons.append((levels[i], multi))
    i+=1

In [ ]:
# Create a dataframe for the extraced multipolygons
kde_df = pd.DataFrame(level_polygons, columns=['level', 'geometry'])

# Convert the dataframe to a geodataframe
kde_geo = gpd.GeoDataFrame(kde_df, geometry='geometry', crs=4326)

# Change the crs for geometric operations
kde_geo = kde_geo.to_crs(epsg=3035)

# Calculate the area of every multipolygon
kde_geo['area'] = kde_geo['geometry'].area

# Change the crs back for visualization
kde_geo = kde_geo.to_crs(4326)

In [ ]:
# Interactive map representation of the estimated densities
kde_geo.explore(column='level', cmap='viridis', legend=False)